In [13]:
from cdx_helper import cdx_scan, CRAWL_CDX

total_bytes = 0
i = 0

for cdx in cdx_scan('https://www.bankofengland.co.uk/', cdx_service=CRAWL_CDX, limit=1000000):
    if cdx.statuscode == '200':
        total_bytes += int(cdx.length)
        i += 1
    if i%1000 == 0:
        print(total_bytes)
    #print(cdx.length, cdx.original, cdx.statuscode)

96159
96159
96159
96159
96159
1663369
52712620
84073715
84073715
84073715
84073715
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
133773698
135822467
141141040
146146265
148374669
148374669
156002034
156002034
158566295
161183911
165615491
170296764
183768574
185168444
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
188584482
190239672
190349846
190349846
190349846
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
190584364
1905843